In [2]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [5]:
train_df=pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\bert\\train.csv")
eval_df=pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\bert\\val.csv")

In [6]:
model_name='indobenchmark/indobert-base-p1'

In [7]:
train_df=train_df[:50010]
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
50005,1383190790147436548,gak,O
50006,1383190790147436548,ada,O
50007,1383190790147436548,kelas,O
50008,1383190790147436548,.,O


In [8]:
eval_df

,sentence_id,words,labels
0,1385573262067847169,Nadiem,B-PER
1,1385573262067847169,Makarim,I-PER
2,1385573262067847169,dg,O
3,1385573262067847169,Presiden,O
4,1385573262067847169,ke-5,O
...,...,...,...
27206,1382913410749190149,aku,O
27207,1383795163412058114,ekonomi,O
27208,1383795163412058114,",",O
27209,1383795163412058114,infrastruktur,O


In [9]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [10]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    # "learning_rate": 4e-5,
    "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [11]:
# model = NERModel('bert', 
#                 'outputs',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# result, model_outputs, wrong_preds = model.eval_model(eval_df)
# result

In [12]:
model = NERModel('bert', 
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Downloading: 100%|██████████| 1.53k/1.53k [00:00<00:00, 1.53MB/s]
Downloading: 100%|██████████| 498M/498M [01:11<00:00, 7.00MB/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Downloading: 100%|██████████| 229k/229k [00:00<00:00, 511kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 112kB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.00kB/s]


In [13]:
model.train_model(train_df,  eval_data=eval_df)

Epoch 5 of 5: 100%|██████████| 5/5 [19:47<00:00, 237.50s/it]


(1655,
 {'global_step': [331, 662, 993, 1324, 1655],
  'precision': [0.40406199893105293,
   0.5975885455915599,
   0.49725877192982454,
   0.5049614112458655,
   0.4662062160828811],
  'recall': [0.31513130471029593,
   0.33055439766569406,
   0.37807419758232597,
   0.381825760733639,
   0.39391413088787],
  'f1_score': [0.3540983606557377,
   0.4256575415995706,
   0.42955245086431443,
   0.4348445288393069,
   0.42702214188883875],
  'train_loss': [0.23670782148838043,
   0.3062273859977722,
   0.3274454176425934,
   0.1139688491821289,
   0.09195061028003693],
  'eval_loss': [0.39475337107108427,
   0.3689894939783171,
   0.3917813096126216,
   0.42396253319699523,
   0.47773986960480813]})

In [14]:
result, model_outputs, wrong_preds = model.eval_model(eval_df)

Running Evaluation: 100%|██████████| 269/269 [00:37<00:00,  7.10it/s]


In [15]:
result

{'eval_loss': 0.47773986960480813,
 'precision': 0.4662062160828811,
 'recall': 0.39391413088787,
 'f1_score': 0.42702214188883875}

In [16]:
# model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

Running Evaluation: 100%|██████████| 269/269 [00:37<00:00,  7.10it/s]


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [19]:
predictions, raw_outputs = model.predict(
    ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"]
        )

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 18.77it/s]


In [20]:
predictions

[[{'@PolsekAlian': 'O'},
  {'PolriBerikan': 'B-ORG'},
  {'inovasi-inovasi': 'O'},
  {'Polri': 'B-ORG'},
  {'yang': 'O'},
  {'memudahkan': 'O'},
  {'masyarakat,': 'O'},
  {'kegiatan': 'O'},
  {'pemolisian': 'O'},
  {'masyarakat': 'O'},
  {'dan': 'O'},
  {'seluruh': 'O'},
  {'tindakan': 'O'},
  {'Kepolisian': 'O'},
  {'dalam': 'O'},
  {'menghadapi': 'O'},
  {'gangguan': 'O'},
  {'Kamtibmas,': 'O'},
  {'dan': 'O'},
  {'lain-lain': 'O'},
  {'-': 'O'},
  {'Jenderal': 'O'},
  {'Polisi': 'O'},
  {'Drs.': 'O'},
  {'Listyo': 'B-PER'},
  {'Sigit': 'I-PER'},
  {'Prabowo,': 'I-PER'},
  {'https://t.co/f8GZ6VPqPD.': 'O'},
  {'-': 'O'},
  {'Kapolri': 'O'},
  {'#PolriTVRadioPresisi': 'O'}]]